### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42349218,77583,79085,51530,0,0,1131,5,2025-03-13 22:00,2025-03-13 22:00,0,0,1782,7,1
34,42350591,106910,79085,3,0,0,1131,6,2025-03-14 18:29,2025-03-14 18:30,0,0,2581,4,1
35,42352471,79085,140269,51530,0,0,1132,0,2025-03-15 22:00,2025-03-15 22:00,3,2,752,1,1
36,42352478,28958,79085,3,0,0,1132,1,2025-03-17 00:29,2025-03-17 00:29,10,0,7309,3,1
37,42354408,6349,79085,51530,0,0,1132,2,2025-03-17 22:00,2025-03-17 22:00,3,0,3281,3,1
38,42354409,79085,6349,51530,0,0,1132,3,2025-03-18 22:00,2025-03-18 22:00,2,1,1055,2,1
39,42355120,120935,79085,3,0,0,1132,4,2025-03-19 21:59,2025-03-19 21:59,0,5,986,5,1
40,42607382,25433,79085,51530,0,0,1132,5,2025-03-20 19:35,2025-03-20 19:35,1,0,4659,5,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

8

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,77583,34547,34439,0,5,1,0,0,max2,9,9,9,42349218
1,79085,35446,35554,1,4,4,3,0,4-4-2c,9,9,10,42349218
0,106910,40277,33449,0,0,3,1,0,Neftçi 3-5-2,6,6,5,42350591
1,79085,29686,36514,1,9,4,1,0,4-4-2c,6,7,6,42350591
0,79085,38230,31856,1,7,3,0,0,4-2-4a,9,9,10,42352471
1,140269,31833,38207,0,11,6,4,1,6!,8,9,9,42352471
0,28958,17884,39161,3,26,1,1,0,ROJO 43,12,8,9,42352478
1,79085,52170,30893,0,1,4,1,0,5-4-1a,5,7,6,42352478
0,6349,25954,39944,2,19,2,1,0,4141 threewheels,12,11,12,42354408
1,79085,44009,30019,0,6,2,0,0,4-4-2c,10,9,10,42354408


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1632

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,89202,0,0,0,0,0,0,0,12,1349
1,10520,0,0,0,0,0,0,0,11,1349
2,23755,0,0,0,0,0,0,0,10,1349
3,79866,0,0,0,0,0,0,0,9,1349
4,78671,0,0,0,0,0,0,0,8,1349
5,89569,0,0,0,0,0,0,0,7,1349
6,124139,0,0,0,0,0,0,0,6,1349
7,122169,0,0,0,0,0,0,0,5,1349
8,79085,0,0,0,0,0,0,0,4,1349
9,80293,0,0,0,0,0,0,0,3,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()